In [ ]:
#Cell_1.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

import random
import math

from sklearn.preprocessing import scale
from sklearn.metrics import mean_squared_error

import warnings
warnings.simplefilter('ignore')

import plotly

# plt.rcParams['font.family'] = 'Yu Mincho'

In [ ]:
#Cell_2.
csv_in="./nikkei.csv"
df=pd.read_csv(csv_in,delimiter="\t")
print(df.shape) #確認
print(df.head()) #確認
print(type(df['日時'][0]))
df['日時']=pd.to_datetime(df['日時'])
print(type(df['日時'][0]))

In [ ]:
#Cell_3.
df_d=df.copy()
df_d['日時']-=df_d['日時'][0]
print(type(df_d['日時'][0]))
df_d['日時']/=np.timedelta64(1,'D')
print(type(df_d['日時'][0]))
print(df_d.head())

In [ ]:
#Cell_4.
print(type(df_d))
df_d=np.asarray(df_d)
print(type(df_d))
print(df_d.shape)
X=df_d[:,:1]
Y=df_d[:,4]
print(X.shape)
print(Y.shape)

In [ ]:
#Cell_5.
import statsmodels.api as sm

X=sm.add_constant(X)
model=sm.OLS(Y,X)
results=model.fit()
print(results.summary())
c=results.params[0]
a=results.params[1]

min_df=min(df_d[:,0])
max_df=max(df_d[:,0])
w_df=max_df-min_df
xx=np.linspace(min_df-0.1*w_df,max_df+0.1*w_df,20)
print(c,a)
#print(xx)
yy=c+a*xx
plt.plot(xx,yy)
plt.plot(df_d[:,0],Y)
plt.show()

In [ ]:
#Cell_7.
obs=Y#df_d['終値']

In [ ]:
#Cell_8.
len(obs)

In [ ]:
#Cell_9.
#https://wagtail.cds.tohoku.ac.jp/coda/python/stochastic-methods/kalman-filter-2.html

dt = 1
F = np.array([[1, dt],
              [0,1]])
G = np.array([[0],
              [math.sqrt(dt)]])
H = np.array([[1,0]])

x = np.zeros((2,1))
xt = np.zeros((2,1))#priorior estimator

#print(xt)

S = np.zeros((2,2))#priorior estimator

R = np.array([[0.01]])
Q = np.array([[0.01]])

x[0,0]=c
x[1,0]=a

#priorior estimators
xt[0,0]=c
xt[1,0]=a

#priorior estimators
S[0,0]=0.01
S[1,1]=0.01

T=[]
X=[]
XT=[]
ZT=[]

t=0
i=0
while i<len(obs):
    D = np.linalg.pinv(H.dot(S.dot(H.T)) + R)
    K = S.dot(H.T).dot(D) #Kalman gain.

    #Filtering step.
    zt = obs[int(i)]
    print('observation is:',zt)
    #posterior estimators.
    x2 = xt + K.dot(zt - H.dot(xt))
    S2 = (np.eye(2) - K.dot(H)).dot(S)

    T.append(t)
    X.append(x[0,0]) #'True' state
    XT.append(x2[0,0]) #estimated state (posterior)
    ZT.append(zt) #observation.

    #prediction step.
    #priorior estimators.
    xt = F.dot(x2)
    S = F.dot(S2.dot(F.T)) + G.dot(Q.dot(G.T))
    print('Predicted next value is:',xt[0,0])
    # update state. This cannot be observed.
    x = F.dot(x) + G.dot( np.random.multivariate_normal([0], R, 1).T )

    t = t + dt
    i+=1

plt.plot(T,X, color='blue', linewidth=1.0, label='State Model')
plt.plot(T,XT, 'o', color='orange', linewidth=1.0, label='Estimation')
plt.plot(T,ZT, '*',color='green',label='Obvervation')
plt.xlabel('T')
plt.ylabel('X')
plt.grid(True)
plt.legend()
plt.show()
